## Titanic 실습 (PCA, XGBoost 이용)
### day 13 실습파일 연계

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Load Dataset

In [2]:
train = pd.read_csv('data/titanic_train.csv', index_col = ["PassengerId"])
print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.info()
# Null 값의 존재 여부 쉽게 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
test = pd.read_csv("data/titanic_test.csv", index_col=["PassengerId"])
print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
Pclass      418 non-null int64
Name        418 non-null object
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Ticket      418 non-null object
Fare        417 non-null float64
Cabin       91 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


### Preprocessing

   ####    encode sex

In [6]:
#train
train.loc[train["Sex"]=="male", "Sex_encode"] = 0 
train.loc[train["Sex"]=="female", "Sex_encode"] = 1

#test
test.loc[test["Sex"]=="male", "Sex_encode"] = 0
test.loc[test["Sex"]=="female", "Sex_encode"] = 1

In [7]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0


In [8]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0


#### encode embarked

In [9]:
# train 
train_embarked = pd.get_dummies(train["Embarked"], prefix="Embk")
print(train_embarked.shape)
train_embarked.head()
print(train.shape)
# concat
train = pd.concat([train, train_embarked], axis=1)
print(train.shape)
train.head()

(891, 3)
(891, 12)
(891, 15)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0,0,0,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,1,0,0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1.0,0,0,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0,0,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0,0,0,1


In [10]:
# test
test_embarked = pd.get_dummies(test["Embarked"], prefix ="Embk")
print(test_embarked.shape)

# concat
test = pd.concat([test,test_embarked], axis=1)
print(test.shape)
test.head()
 # survive 필드가 없어서 training set 과 열 개수가 다름

(418, 3)
(418, 14)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0,0,1,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0,0,0,1
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0,0,1,0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0,0,0,1
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0,0,0,1


In [11]:
train.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'Sex_encode', 'Embk_C', 'Embk_Q',
       'Embk_S'],
      dtype='object')

In [12]:
test.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'Sex_encode', 'Embk_C', 'Embk_Q', 'Embk_S'],
      dtype='object')

### Null checking - e.g Fare fill in 

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 15 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           714 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Ticket        891 non-null object
Fare          891 non-null float64
Cabin         204 non-null object
Embarked      889 non-null object
Sex_encode    891 non-null float64
Embk_C        891 non-null uint8
Embk_Q        891 non-null uint8
Embk_S        891 non-null uint8
dtypes: float64(3), int64(4), object(5), uint8(3)
memory usage: 93.1+ KB


In [22]:
# Feature 로 사용될 특성에 대해서만 Null 값이 있는지 여부를 체크하면 되나 ? 
# 예를 들어 Cabin 필드는 Null 값이 많은데... 사용하지 않으니 전처리 필요 X ? 

train[train['Fare'].isnull()]
train[train['Age'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,,
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0.0,0,1,0
18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,0.0,0,0,1
20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,1.0,1,0,0
27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,0.0,1,0,0
29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,1.0,0,1,0
30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S,0.0,0,0,1
32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,B78,C,1.0,1,0,0
33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q,1.0,0,1,0
37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C,0.0,1,0,0


In [23]:
mean_age = train.loc[train['Age'].notnull(), 'Age'].mean()
train['Age'].fillna(mean_age, inplace=True)

In [25]:
train[train['Age'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,,


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 14 columns):
Pclass        418 non-null int64
Name          418 non-null object
Sex           418 non-null object
Age           332 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Ticket        418 non-null object
Fare          417 non-null float64
Cabin         91 non-null object
Embarked      418 non-null object
Sex_encode    418 non-null float64
Embk_C        418 non-null uint8
Embk_Q        418 non-null uint8
Embk_S        418 non-null uint8
dtypes: float64(3), int64(3), object(5), uint8(3)
memory usage: 40.4+ KB


In [15]:
test[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0.0,0,0,1


In [18]:
mean_fare = test.loc[test["Fare"].notnull(), "Fare"].mean()
test['Fare'].fillna(mean_fare, inplace=True)

In [19]:
test[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,


In [26]:
mean_age = test.loc[test['Age'].notnull(), 'Age'].mean()
test['Age'].fillna(mean_age, inplace=True)

In [27]:
test[test['Age'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,,,,,,


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 15 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Ticket        891 non-null object
Fare          891 non-null float64
Cabin         204 non-null object
Embarked      889 non-null object
Sex_encode    891 non-null float64
Embk_C        891 non-null uint8
Embk_Q        891 non-null uint8
Embk_S        891 non-null uint8
dtypes: float64(3), int64(4), object(5), uint8(3)
memory usage: 93.1+ KB


In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 14 columns):
Pclass        418 non-null int64
Name          418 non-null object
Sex           418 non-null object
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Ticket        418 non-null object
Fare          418 non-null float64
Cabin         91 non-null object
Embarked      418 non-null object
Sex_encode    418 non-null float64
Embk_C        418 non-null uint8
Embk_Q        418 non-null uint8
Embk_S        418 non-null uint8
dtypes: float64(3), int64(3), object(5), uint8(3)
memory usage: 40.4+ KB


### Train

In [31]:
feature_names = ["Age", "Pclass", "Sex_encode", "Fare", "SibSp", "Parch",
                 "Embk_C", "Embk_Q", "Embk_S"]

X_train = train[feature_names]
print(X_train.shape)
X_train.head()

(891, 9)


,Age,Pclass,Sex_encode,Fare,SibSp,Parch,Embk_C,Embk_Q,Embk_S
PassengerId,,,,,,,,,
1,22.0,3,0.0,7.2500,1,0,0,0,1
2,38.0,1,1.0,71.2833,1,0,1,0,0
3,26.0,3,1.0,7.9250,0,0,0,0,1
4,35.0,1,1.0,53.1000,1,0,0,0,1
5,35.0,3,0.0,8.0500,0,0,0,0,1


In [32]:
label_name = "Survived"
y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### Scaling 

#### MinMaxScaler

In [45]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)

# X_train_scaled = scaler.transform(X_train)
# # X_test_scaled = model.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


#### StandardScaler

In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


### Modeling

#### SVM

In [91]:
# from sklearn.svm import SVC
# model = SVC(C=100)
# model.fit(X_train_scaled, y_train)

# # model.score(X_test_scaled)

#### XGBoost Regressor

In [92]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

In [93]:
# model = xgb.XGBRegressor(colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=10)
# model.fit(X_train_scaled, y_train)

#### XGBooost Classifier

In [94]:
# import xgboost as xgb
# model = xgb.XGBClassifier(colsample_byte=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=10)
# model.fit(X_train_scaled, y_train)

### Predict

In [95]:
X_test = test[feature_names]
X_test_scaled = scaler.transform(X_test)
print(X_test.shape)
X_test

# Null 이 있는 Cell 찾기 
null_columns=X_test.columns[X_test.isnull().any()]
# X_test.columns[test.isnull().any()]
print(X_test[X_test.isnull().any(axis=1)][null_columns].head())

(418, 9)
Empty DataFrame
Columns: []
Index: []


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


In [96]:
prediction = model.predict(X_test_scaled)

print(prediction.shape)

(418,)


# PCA

In [106]:
from sklearn.decomposition import PCA

#### scaling for PCA

In [107]:
pca = PCA(n_components=2, whiten=True, random_state=0).fit(X_train)
X_train_pca = pca.transform(X_train)

print(X_train_pca.shape)

(891, 2)


#### modeling for PCA  by kNN

In [110]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train_pca, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

#### Predict for PCA

In [113]:
X_test = test[feature_names]
X_test_pca = pca.transform(X_test)

prediction = model.predict(X_test_pca)

print(prediction.shape)

(418,)


### Submit

In [114]:
submission = pd.read_csv("data/gender_submission.csv", index_col="PassengerId")

submission["Survived"] = prediction.astype(np.int32)

print(submission.shape)
submission.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,1
895,1
896,0


In [115]:
submission.to_csv("baseline-script.csv")